In [1]:
import pandas as pd
from pathlib import Path
import xlwings as xw
import psutil
import os


import tkinter as tk
from tkinter import simpledialog

# Tkinter 창 생성
root = tk.Tk()
root.withdraw()  # 메인 창 숨기기

# 사용자로부터 입력 받기
조회일 = simpledialog.askstring("Input", "업로드 기준일을 입력하세요 YYYYMMDD 타입 입니다.:")





input_folder = 'c:/주요계정' # 원본 데이터 폴더
raw_data_dir = Path(input_folder)
excel_files = raw_data_dir.glob('*rpa*')  # 폴더 내 데이터 파일 이름 

total_df = pd.DataFrame() # 빈 DataFrame 생성
print(excel_files)



# print(str(조회일)+"기준으로 조회합니다")



for excel_file in excel_files:
    app = xw.App()
    
    wb = xw.Book(excel_file)
    sht=wb.sheets[0]
        #파일명어떻게 할지 
    df = sht.range('a1').options(pd.DataFrame,expand='table',Header=1).value
    wb.close()

    
    
    
#     for proc in psutil.process_iter():

#         try:
#             # 프로세스 이름을 가져옵니다.
#             process_name = proc.name()

#             # 만약 엑셀이 실행 중이면,
#             if 'EXCEL' in process_name.upper():
#                 # 프로세스를 종료합니다.
#                 os.kill(proc.pid, 9)
#                 print(f"{process_name} 프로세스를 종료했습니다.")

#         except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
#             # 종료할 수 없는 프로세스는 건너뜁니다.
#             pass

#     print("열린 엑셀파일을 모두 종료 했습니다")
   
    

    df = df.reset_index()
    selected_columns = ['회사코드','G/L 계정 설명','손익 센터','손익센터명','그룹통화 잔액']
    df = df[selected_columns]



    
    df3 = df.reset_index()
    del_row = df3.dropna(subset=['회사코드'])
    df4 =  del_row.fillna("비어있음")

    grouped = df4.groupby(['회사코드','G/L 계정 설명','손익 센터'])[['그룹통화 잔액']].sum()
    grouped2 = grouped.reset_index()

    total_df = total_df.append(grouped2, ignore_index= True)
    print("파일 통합을 시작 합니다")
    
    

total_df # 통합한 DataFrame 데이터 출력
total_df.to_excel('c:/주요계정/작성.xlsx',sheet_name='sheet1')
print("파일을 하나로 병합했습니다")




import numpy as np
import pathlib as Path





wb = xw.Book('c:/주요계정/작성.xlsx')
sht=wb.sheets['sheet1']
df = sht.range('a1').options(pd.DataFrame,expand='table',index=False).value


df['손익 센터'] = df['손익 센터'].replace('AAID1903','AAID1902') #손익센터변경

df['손익 센터'] = df['손익 센터'].replace('TBEMY1601','BEMY1601')

df['손익 센터'] = df['손익 센터'].replace('TAAMY1701','AAMY1701')

df['손익 센터'] = df['손익 센터'].replace('TAAMY1701','AAMY1701')

df['손익 센터'] = df['손익 센터'].replace('TAAID2102','AAID2102')

df['손익 센터'] = df['손익 센터'].replace('TAAID2101','AAID1902')

df['손익 센터'] = df['손익 센터'].replace('AAPL2101','TAAPL2101')

df['손익 센터'] = df['손익 센터'].replace('TBEPH1402','BEPH1402')

df['손익 센터'] = df['손익 센터'].replace('TBEID1401','BEID1401')

df['손익 센터'] = df['손익 센터'].replace('ANUS2201','TANUS2201')

df['손익 센터'] = df['손익 센터'].replace('AOUS2201','TAOUS2201')

df['손익 센터'] = df['손익 센터'].replace('ANID2201','ANID2202')

df['손익 센터'] = df['손익 센터'].replace('ANID2203','ANID2201')





print("통합파일을 연결기준으로 변환합니다")

wb = xw.Book('reference3.xlsx') # 계정과목표 불러오기 df2
sht=wb.sheets['sheet1']
df1 = sht.range('a1').options(pd.DataFrame,expand='table',index=False).value
df1
print(df1.dtypes)
df2 = df1.astype('object')

print(df2.dtypes)
df2




merge2 = pd.merge(df,df2,how="left",on="G/L 계정 설명") #원본에 구분 추가

dfp2=pd.pivot_table(merge2,index=["손익 센터","구분"],values="그룹통화 잔액",aggfunc='sum')

dfp2["구분손익센터"]= np.nan

dfp2 = dfp2.reset_index()
dfp2.info()
dfp2['구분손익센터'] = dfp2['손익 센터']+dfp2['구분']

folder='c:/주요계정'

excel_file=folder+'7.xlsx'

dfp2.to_excel(excel_file,sheet_name='pivot_table') #원본에 문자병합

dfp2.to_excel('c:/주요계정/중간.xlsx',sheet_name='pot_table')




wb = xw.Book('reference.xlsx') 
sht=wb.sheets['sheet1']
refer = sht.range('a1').options(pd.DataFrame,expand='table',index=False).value
refer['구분손익센터'] = refer['손익 센터']+refer['계정']
refer = refer.drop('손익 센터', axis=1)

refer # refer에 병합





merge3 = pd.merge(dfp2,refer,how="left",on="구분손익센터") #원본에 구분 추가
merge3

merge3.to_excel('c:/주요계정/중간11.xlsx',sheet_name='pot_table') #테스트







merge3["연결후"]= np.nan
merge3['연결조정'] = merge3['연결조정'].fillna(0)
merge3['연결후'] = merge3['그룹통화 잔액']+merge3['연결조정']
zero_drop = merge3['연결후'] != 0 
merge4 = merge3[zero_drop ]





# 조건부로직을 사용하여 B 칼럼 값에 -1을 곱함
merge4['연결후'] = merge4.apply(lambda row: row['연결후'] * -1 if str(row['구분']).startswith('초') else row['연결후'], axis=1)
merge4['연결후'] = merge4.apply(lambda row: row['연결후'] * -1 if str(row['구분']).startswith('선수') else row['연결후'], axis=1)
merge4['연결후'] = merge4.apply(lambda row: row['연결후'] * -1 if str(row['구분']).startswith('매입채무') else row['연결후'], axis=1)





merge4 = merge4.reindex(columns=['손익 센터','연결후','기간','구분'])




# 문자열로 처리 후 날짜로 변환


date_value = pd.to_datetime(조회일, format='%Y%m%d').strftime('%Y-%m-%d')




merge4['기간'] = date_value


merge4.to_excel('c:/주요계정/result.xlsx',sheet_name='pot_table')

print("연결기준 계정 변경 완료")


import pyodbc



# MS Access 데이터베이스에 연결합니다.
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=d:\archive\손익\주요계정.accdb;')


merge4 =merge4.rename(columns={'손익 센터':'손익센터'})
print("MS Access에 연결합니다")


cursor = conn.cursor()
for index, row in merge4.iterrows():
    cursor.execute("INSERT INTO 주요계정 (손익센터,금액,기간,계정) VALUES (?,?,?,?)",
                   row['손익센터'], row['연결후'], row['기간'],row['구분'])
conn.commit()

# MS Access 데이터베이스 연결을 닫습니다.
conn.close()
print("Access 업로드를 완료하였습니다.")

<generator object Path.glob at 0x0000022342CF86D0>


C:\Users\hec\AppData\Local\Temp\ipykernel_10468\418671494.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(grouped2, ignore_index= True)


파일 통합을 시작 합니다


C:\Users\hec\AppData\Local\Temp\ipykernel_10468\418671494.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(grouped2, ignore_index= True)


파일 통합을 시작 합니다


...

파일 통합을 시작 합니다


C:\Users\hec\AppData\Local\Temp\ipykernel_10468\418671494.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_df = total_df.append(grouped2, ignore_index= True)


파일 통합을 시작 합니다
파일을 하나로 병합했습니다
통합파일을 연결기준으로 변환합니다
계정 번호        float64
G/L 계정 설명     object
구분            object
dtype: object
계정 번호        object
G/L 계정 설명    object
구분           object
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2733 entries, 0 to 2732
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   손익 센터    2733 non-null   object 
 1   구분       2733 non-null   object 
 2   그룹통화 잔액  2733 non-null   float64
 3   구분손익센터   0 non-null      float64
dtypes: float64(2), object(2)
memory usage: 85.5+ KB


C:\Users\hec\AppData\Local\Temp\ipykernel_10468\418671494.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge4['연결후'] = merge4.apply(lambda row: row['연결후'] * -1 if str(row['구분']).startswith('초') else row['연결후'], axis=1)
C:\Users\hec\AppData\Local\Temp\ipykernel_10468\418671494.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge4['연결후'] = merge4.apply(lambda row: row['연결후'] * -1 if str(row['구분']).startswith('선수') else row['연결후'], axis=1)
C:\Users\hec\AppData\Local\Temp\ipykernel_10468\4186714

연결기준 계정 변경 완료
MS Access에 연결합니다
Access 업로드를 완료하였습니다.


In [9]:
import pandas as pd
from pathlib import Path
import xlwings as xw
import psutil
import os
import tkinter as tk
from tkinter import simpledialog

# Tkinter 창 생성
root = tk.Tk()
root.withdraw()  # 메인 창 숨기기

# 사용자로부터 입력 받기
조회일 = simpledialog.askstring("Input", "업로드 기준일을 입력하세요 YYYYMMDD 타입 입니다.:")

input_folder = 'c:/주요계정'  # 원본 데이터 폴더
raw_data_dir = Path(input_folder)
excel_files = raw_data_dir.glob('*rpa*')  # 폴더 내 데이터 파일 이름

total_df = pd.DataFrame()  # 빈 DataFrame 생성
print(list(excel_files))  # 파일 목록 출력

for excel_file in excel_files:
    app = xw.App(visible=False)
    
    wb = xw.Book(excel_file)
    sht = wb.sheets[0]
    
    # 파일의 데이터를 DataFrame으로 변환
    df = sht.range('A1').options(pd.DataFrame, expand='table', header=1).value
    
    # 필요한 열만 선택
    selected_columns = ['회사코드', 'G/L 계정 설명', '손익 센터', '손익센터명', '그룹통화 잔액']
    if set(selected_columns).issubset(df.columns):
        df = df[selected_columns]
    
    df = df.reset_index()
    wb.close()
    app.quit()
    
    df3 = df.reset_index()
    del_row = df3.dropna(subset=['회사코드'])
    df4 = del_row.fillna("비어있음")

    grouped = df4.groupby(['회사코드', 'G/L 계정 설명', '손익 센터'])[['그룹통화 잔액']].sum()
    grouped2 = grouped.reset_index()

    total_df = total_df.append(grouped2, ignore_index=True)
    print("파일 통합을 시작합니다")

total_df  # 통합한 DataFrame 데이터 출력
total_df.to_excel('c:/주요계정/작성.xlsx', sheet_name='sheet1')
print("파일을 하나로 병합했습니다")

wb = xw.Book('c:/주요계정/작성.xlsx')
sht = wb.sheets['sheet1']
df = sht.range('A1').options(pd.DataFrame, expand='table', index=False).value

df['손익 센터'] = df['손익 센터'].replace('AAID1903', 'AAID1902')  # 손익센터 변경
df['손익 센터'] = df['손익 센터'].replace('TBEMY1601', 'BEMY1601')
df['손익 센터'] = df['손익 센터'].replace('TAAMY1701', 'AAMY1701')
df['손익 센터'] = df['손익 센터'].replace('TAAID2102', 'AAID2102')
df['손익 센터'] = df['손익 센터'].replace('TAAID2101', 'AAID1902')
df['손익 센터'] = df['손익 센터'].replace('AAPL2101', 'TAAPL2101')
df['손익 센터'] = df['손익 센터'].replace('TBEPH1402', 'BEPH1402')
df['손익 센터'] = df['손익 센터'].replace('TBEID1401', 'BEID1401')
df['손익 센터'] = df['손익 센터'].replace('ANUS2201', 'TANUS2201')
df['손익 센터'] = df['손익 센터'].replace('AOUS2201', 'TAOUS2201')
df['손익 센터'] = df['손익 센터'].replace('ANID2201', 'ANID2202')
df['손익 센터'] = df['손익 센터'].replace('ANID2203', 'ANID2201')

print("통합파일을 연결기준으로 변환합니다")

wb = xw.Book('reference3.xlsx')  # 계정과목표 불러오기 df2
sht = wb.sheets['sheet1']
df1 = sht.range('A1').options(pd.DataFrame, expand='table', index=False).value
df1
print(df1.dtypes)
df2 = df1.astype('object')

print(df2.dtypes)
df2

merge2 = pd.merge(df, df2, how="left", on="G/L 계정 설명")  # 원본에 구분 추가

dfp2 = pd.pivot_table(merge2, index=["손익 센터", "구분"], values="그룹통화 잔액", aggfunc='sum')
dfp2["구분손익센터"] = np.nan

dfp2 = dfp2.reset_index()
dfp2.info()
dfp2['구분손익센터'] = dfp2['손익 센터'] + dfp2['구분']

folder = 'c:/주요계정'

excel_file = folder + '7.xlsx'

dfp2.to_excel(excel_file, sheet_name='pivot_table')  # 원본에 문자병합

dfp2.to_excel('c:/주요계정/중간.xlsx', sheet_name='pot_table')

wb = xw.Book('reference.xlsx')
sht = wb.sheets['sheet1']
refer = sht.range('A1').options(pd.DataFrame, expand='table', index=False).value
refer['구분손익센터'] = refer['손익 센터'] + refer['계정']
refer = refer.drop('손익 센터', axis=1)

refer  # refer에 병합

merge3 = pd.merge(dfp2, refer, how="left", on="구분손익센터")  # 원본에 구분 추가
merge3

merge3.to_excel('c:/주요계정/중간11.xlsx', sheet_name='pot_table')  # 테스트

merge3["연결후"] = np.nan
merge3['연결조정'] = merge3['연결조정'].fillna(0)
merge3['연결후'] = merge3['그룹통화 잔액'] + merge3['연결조정']
zero_drop = merge3['연결후'] != 0
merge4 = merge3[zero_drop]

# 조건부 로직을 사용하여 B 칼럼 값에 -1을 곱함
merge4['연결후'] = merge4.apply(lambda row: row['연결후'] * -1 if str(row['구분']).startswith('초') else row['연결후'], axis=1)
merge4['연결후'] = merge4.apply(lambda row: row['연결후'] * -1 if str(row['구분']).startswith('선수') else row['연결후'], axis=1)
merge4['연결후'] = merge4.apply(lambda row: row['연결후'] * -1 if str(row['구분']).startswith('매입채무') else row['연결후'], axis=1)

merge4 = merge4.reindex(columns=['손익 센터', '연결후', '기간', '구분'])

# 문자열로 처리 후 날짜로 변환
date_value = pd.to_datetime(조회일, format='%Y%m%d').strftime('%Y-%m-%d')

merge4['기간'] = date_value

merge4.to_excel('c:/주요계정/result.xlsx', sheet_name='pot_table')

print("연결기준 계정 변경 완료")

import pyodbc

# MS Access 데이터베이스에 연결합니다.
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=d:\archive\손익\주요계정.accdb;')

merge4 = merge4.rename(columns={'손익 센터': '손익센터'})
print("MS Access에 연결합니다")

cursor = conn.cursor()
for index, row in merge4.iterrows():
    cursor.execute("INSERT INTO 주요계정 (손익센터, 금액, 기간, 계정) VALUES (?,?,?,?)",
                   row['손익센터'], row['연결후'], row['기간'], row['구분'])
conn.commit()

# MS Access 데이터베이스 연결을 닫습니다.
conn.close()
print("Access 업로드를 완료하였습니다.")


[WindowsPath('c:/주요계정/rpa 21030199 20230731.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20230831.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20230930.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20231031.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20231130.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20231231.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20240131.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20240731.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020141231.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020150331.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020150630.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020150930.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020151231.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020160331.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020160630.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020160930.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020161231.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020170331.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020170630.0.XLSX'), WindowsP

KeyError: '손익 센터'

In [10]:
import pandas as pd
from pathlib import Path
import xlwings as xw
import psutil
import os
import tkinter as tk
from tkinter import simpledialog

# Tkinter 창 생성
root = tk.Tk()
root.withdraw()  # 메인 창 숨기기

# 사용자로부터 입력 받기
조회일 = simpledialog.askstring("Input", "업로드 기준일을 입력하세요 YYYYMMDD 타입 입니다.:")

input_folder = 'c:/주요계정'  # 원본 데이터 폴더
raw_data_dir = Path(input_folder)
excel_files = list(raw_data_dir.glob('*rpa*'))  # 폴더 내 데이터 파일 이름

total_df = pd.DataFrame()  # 빈 DataFrame 생성
print(excel_files)  # 파일 목록 출력

for excel_file in excel_files:
    app = xw.App(visible=False)
    wb = xw.Book(excel_file)
    sht = wb.sheets[0]

    # 파일의 데이터를 DataFrame으로 변환
    df = sht.range('A1').options(pd.DataFrame, expand='table', header=1).value
    
    # 필요한 열만 선택
    selected_columns = ['회사코드', 'G/L 계정 설명', '손익 센터', '손익센터명', '그룹통화 잔액']
    if set(selected_columns).issubset(df.columns):
        df = df[selected_columns]
    
        df = df.reset_index(drop=True)
        wb.close()
        app.quit()

        df = df.reset_index(drop=True)
        del_row = df.dropna(subset=['회사코드'])
        df4 = del_row.fillna("비어있음")

        grouped = df4.groupby(['회사코드', 'G/L 계정 설명', '손익 센터'])[['그룹통화 잔액']].sum()
        grouped2 = grouped.reset_index()

        total_df = total_df.append(grouped2, ignore_index=True)
        print("파일 통합을 시작합니다")
    else:
        wb.close()
        app.quit()
        print(f"{excel_file}에 필요한 열이 없습니다: {set(selected_columns) - set(df.columns)}")

# 통합한 DataFrame 데이터 출력
print(total_df)
total_df.to_excel('c:/주요계정/작성.xlsx', sheet_name='sheet1')
print("파일을 하나로 병합했습니다")

# 변경된 통합파일 로드
wb = xw.Book('c:/주요계정/작성.xlsx')
sht = wb.sheets['sheet1']
df = sht.range('A1').options(pd.DataFrame, expand='table', index=False).value

# 손익 센터 값 변경
if '손익 센터' in df.columns:
    df['손익 센터'] = df['손익 센터'].replace('AAID1903', 'AAID1902')
    df['손익 센터'] = df['손익 센터'].replace('TBEMY1601', 'BEMY1601')
    df['손익 센터'] = df['손익 센터'].replace('TAAMY1701', 'AAMY1701')
    df['손익 센터'] = df['손익 센터'].replace('TAAID2102', 'AAID2102')
    df['손익 센터'] = df['손익 센터'].replace('TAAID2101', 'AAID1902')
    df['손익 센터'] = df['손익 센터'].replace('AAPL2101', 'TAAPL2101')
    df['손익 센터'] = df['손익 센터'].replace('TBEPH1402', 'BEPH1402')
    df['손익 센터'] = df['손익 센터'].replace('TBEID1401', 'BEID1401')
    df['손익 센터'] = df['손익 센터'].replace('ANUS2201', 'TANUS2201')
    df['손익 센터'] = df['손익 센터'].replace('AOUS2201', 'TAOUS2201')
    df['손익 센터'] = df['손익 센터'].replace('ANID2201', 'ANID2202')
    df['손익 센터'] = df['손익 센터'].replace('ANID2203', 'ANID2201')
else:
    print("손익 센터 열이 없습니다.")

print("통합파일을 연결기준으로 변환합니다")

wb = xw.Book('reference3.xlsx')  # 계정과목표 불러오기 df2
sht = wb.sheets['sheet1']
df1 = sht.range('A1').options(pd.DataFrame, expand='table', index=False).value
df1
print(df1.dtypes)
df2 = df1.astype('object')

print(df2.dtypes)
df2

merge2 = pd.merge(df, df2, how="left", on="G/L 계정 설명")  # 원본에 구분 추가

dfp2 = pd.pivot_table(merge2, index=["손익 센터", "구분"], values="그룹통화 잔액", aggfunc='sum')
dfp2["구분손익센터"] = np.nan

dfp2 = dfp2.reset_index()
dfp2.info()
dfp2['구분손익센터'] = dfp2['손익 센터'] + dfp2['구분']

folder = 'c:/주요계정'

excel_file = folder + '7.xlsx'

dfp2.to_excel(excel_file, sheet_name='pivot_table')  # 원본에 문자병합

dfp2.to_excel('c:/주요계정/중간.xlsx', sheet_name='pot_table')

wb = xw.Book('reference.xlsx')
sht = wb.sheets['sheet1']
refer = sht.range('A1').options(pd.DataFrame, expand='table', index=False).value
refer['구분손익센터'] = refer['손익 센터'] + refer['계정']
refer = refer.drop('손익 센터', axis=1)

refer  # refer에 병합

merge3 = pd.merge(dfp2, refer, how="left", on="구분손익센터")  # 원본에 구분 추가
merge3

merge3.to_excel('c:/주요계정/중간11.xlsx', sheet_name='pot_table')  # 테스트

merge3["연결후"] = np.nan
merge3['연결조정'] = merge3['연결조정'].fillna(0)
merge3['연결후'] = merge3['그룹통화 잔액'] + merge3['연결조정']
zero_drop = merge3['연결후'] != 0
merge4 = merge3[zero_drop]

# 조건부 로직을 사용하여 B 칼럼 값에 -1을 곱함
merge4['연결후'] = merge4.apply(lambda row: row['연결후'] * -1 if str(row['구분']).startswith('초') else row['연결후'], axis=1)
merge4['연결후'] = merge4.apply(lambda row: row['연결후'] * -1 if str(row['구분']).startswith('선수') else row['연결후'], axis=1)
merge4['연결후'] = merge4.apply(lambda row: row['연결후'] * -1 if str(row['구분']).startswith('매입채무') else row['연결후'], axis=1)

merge4 = merge4.reindex(columns=['손익 센터', '연결후', '기간', '구분'])

# 문자열로 처리 후 날짜로 변환
date_value = pd.to_datetime(조회일, format='%Y%m%d').strftime('%Y-%m-%d')

merge4['기간'] = date_value

merge4.to_excel('c:/주요계정/result.xlsx', sheet_name='pot_table')

print("연결기준 계정 변경 완료")

import pyodbc

# MS Access 데이터베이스에 연결합니다.
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=d:\archive\손익\주요계정.accdb;')

merge4 = merge4.rename(columns={'손익 센터': '손익센터'})
print("MS Access에 연결합니다")

cursor = conn.cursor()
for index, row in merge4.iterrows():
    cursor.execute("INSERT INTO 주요계정 (손익센터, 금액, 기간, 계정) VALUES (?,?,?,?)",
                   row['손익센터'], row['연결후'], row['기간'], row['구분'])
conn.commit()

# MS Access 데이터베이스 연결을 닫습니다.
conn.close()
print("Access 업로드를 완료하였습니다.")


[WindowsPath('c:/주요계정/rpa 21030199 20230731.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20230831.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20230930.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20231031.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20231130.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20231231.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20240131.XLSX'), WindowsPath('c:/주요계정/rpa 21030199 20240731.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020141231.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020150331.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020150630.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020150930.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020151231.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020160331.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020160630.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020160930.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020161231.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020170331.0.XLSX'), WindowsPath('c:/주요계정/rpa21030199.020170630.0.XLSX'), WindowsP

KeyboardInterrupt: 

In [3]:
import pandas as pd
import xlwings as xw
from trino.dbapi import connect
from trino.auth import BasicAuthentication
import tkinter as tk
from tkinter import messagebox
from datetime import datetime

# Trino DB 연결 설정 함수
def get_connection():
    return connect(
        host='data-query.hec.co.kr',
        port=443,
        http_scheme='https',
        user='0900051',
        auth=BasicAuthentication('0900051', 'roseofpA1!!!!!'),
        catalog='dw',
        schema='fie'
    )

# DB에서 데이터 가져오기 함수
def get_filtered_data(conn, end_date):
    query = f"""
    SELECT v.gc, v.profit_center
    FROM s_accounting_ledger v
    WHERE v.acc_num BETWEEN 11530110 AND 11530199
      AND v.posting_date <= CAST('{end_date.strftime("%Y-%m-%d")}' AS DATE)
    """
    df = pd.read_sql(query, conn)
    df['계정'] = '매출채권'  # '계정' 칼럼 추가
    return df

# Profit center 값 변경 함수
def update_profit_center(df):
    replacements = {
        'AAID1903': 'AAID1902',
        'TBEMY1601': 'BEMY1601',
        'TAAMY1701': 'AAMY1701',
        'TAAID2102': 'AAID2102',
        'TAAID2101': 'AAID1902',
        'AAPL2101': 'TAAPL2101',
        'TBEPH1402': 'BEPH1402',
        'TBEID1401': 'BEID1401',
        'ANUS2201': 'TANUS2201',
        'AOUS2201': 'TAOUS2201',
        'ANID2201': 'ANID2202',
        'ANID2203': 'ANID2201'
    }
    
    df['profit_center'] = df['profit_center'].replace(replacements)
    return df

# Excel 파일과 병합하는 함수
def merge_with_excel(df):
    # 엑셀 파일 열기
    excel_path = 'refe.xlsx'  # 기본 폴더에 있는 파일 경로
    app = xw.App(visible=False)
    wb = app.books.open(excel_path)
    sheet = wb.sheets['Sheet1']
    
    # Excel 데이터 불러오기 (인덱스 없이 읽기)
    excel_df = sheet.range('A1').options(pd.DataFrame, index=False, expand='table').value
    
    # 중간 출력: 엑셀에서 읽어온 데이터 출력
    print("Excel에서 읽어온 데이터:")
    print(excel_df)
    
    # ad_amount 컬럼명을 gc로 변경
    excel_df = excel_df.rename(columns={'ad_amount': 'gc'})
    
    # 두 DataFrame 병합 (아래로 붙이기)
    merged_df = pd.concat([df, excel_df], ignore_index=True)
    
    # 엑셀 파일 닫기
    wb.close()
    app.quit()
    
    return merged_df

# Group by 하고 gc 합계 계산
def group_and_sum_gc(df):
    df_grouped = df.groupby('profit_center', as_index=False)['gc'].sum()
    return df_grouped

# 피벗 테이블 생성 함수
def create_pivot_table(df):
    # xlwings를 사용하여 피벗 테이블 생성
    app = xw.App(visible=True)
    wb = app.books.add()
    sheet = wb.sheets[0]
    
    # 데이터 삽입
    sheet.range('A1').value = df
    
    # 피벗 테이블 범위 설정
    pivot_sheet = wb.sheets.add('피벗테이블')
    data_range = sheet.range('A1').expand()
    
    # 피벗 테이블 생성
    pivot_cache = pivot_sheet.book.api.PivotCaches().Create(SourceType=xw.constants.PivotTableSourceType.xlDatabase,
                                                            SourceData=data_range.api)
    pivot_table = pivot_cache.CreatePivotTable(TableDestination=pivot_sheet.range('A3').api, TableName='PivotTable1')
    
    # 피벗 테이블 설정 (profit_center 기준, gc 합계)
    pivot_table.PivotFields('profit_center').Orientation = xw.constants.PivotFieldOrientation.xlRowField
    pivot_table.PivotFields('gc').Orientation = xw.constants.PivotFieldOrientation.xlDataField
    
    # 피벗 테이블 저장
    wb.save('피벗_결과.xlsx')
    app.quit()

# 메인 실행 함수
def main_process(end_date):
    conn = None
    try:
        # DB 연결
        conn = get_connection()
        
        # DB에서 필터링된 데이터 가져오기
        df = get_filtered_data(conn, end_date)
        
        # Profit center 값 변경
        df = update_profit_center(df)
        
        # Excel 파일과 병합
        merged_df = merge_with_excel(df)
        
        # Group by 하고 gc 합계 계산
        grouped_df = group_and_sum_gc(merged_df)
        
        # 피벗 테이블 생성
        create_pivot_table(grouped_df)

    except Exception as e:
        print(f"에러 발생: {str(e)}")
    finally:
        if conn:
            conn.close()

# Tkinter GUI로 사용자 입력 받기
def run_gui():
    def process_data():
        input_date = entry_date.get()
        try:
            end_date = datetime.strptime(input_date, "%Y-%m-%d")
            main_process(end_date)
            messagebox.showinfo("성공", "작업이 완료되었습니다!")
        except ValueError:
            messagebox.showerror("오류", "잘못된 날짜 형식입니다. YYYY-MM-DD 형식으로 입력해주세요.")
    
    root = tk.Tk()
    root.title("날짜 입력")
    
    label_date = tk.Label(root, text="기준 날짜 (YYYY-MM-DD):")
    label_date.pack(pady=10)
    
    entry_date = tk.Entry(root)
    entry_date.pack(pady=10)
    
    btn_process = tk.Button(root, text="실행", command=process_data)
    btn_process.pack(pady=10)
    
    root.mainloop()

# GUI 실행
run_gui()


In [4]:
import pandas as pd
import xlwings as xw
from trino.dbapi import connect
from trino.auth import BasicAuthentication
import tkinter as tk
from tkinter import messagebox
from datetime import datetime

# Trino DB 연결 설정 함수
def get_connection():
    return connect(
        host='data-query.hec.co.kr',
        port=443,
        http_scheme='https',
        user='0900051',
        auth=BasicAuthentication('0900051', 'roseofpA1!!!!!'),
        catalog='dw',
        schema='fie'
    )

# DB에서 데이터 가져오기 함수
def get_filtered_data(conn, end_date):
    query = f"""
    SELECT v.gc, v.profit_center
    FROM s_accounting_ledger v
    WHERE v.acc_num BETWEEN 11530110 AND 11530199
      AND v.posting_date <= CAST('{end_date.strftime("%Y-%m-%d")}' AS DATE)
    """
    df = pd.read_sql(query, conn)
    df['계정'] = '매출채권'  # '계정' 칼럼 추가
    return df

# Profit center 값 변경 함수
def update_profit_center(df):
    replacements = {
        'AAID1903': 'AAID1902',
        'TBEMY1601': 'BEMY1601',
        'TAAMY1701': 'AAMY1701',
        'TAAID2102': 'AAID2102',
        'TAAID2101': 'AAID1902',
        'AAPL2101': 'TAAPL2101',
        'TBEPH1402': 'BEPH1402',
        'TBEID1401': 'BEID1401',
        'ANUS2201': 'TANUS2201',
        'AOUS2201': 'TAOUS2201',
        'ANID2201': 'ANID2202',
        'ANID2203': 'ANID2201'
    }
    
    df['profit_center'] = df['profit_center'].replace(replacements)
    return df

# Excel 파일과 병합하는 함수
def merge_with_excel(df):
    # 엑셀 파일 열기
    excel_path = 'refe.xlsx'  # 기본 폴더에 있는 파일 경로
    app = xw.App(visible=True)
    wb = app.books.open(excel_path)
    sheet = wb.sheets['Sheet1']
    
    # Excel 데이터 불러오기 (첫 줄을 헤더로 설정)
    excel_df = sheet.range('A1').options(pd.DataFrame, index=False, header=True, expand='table').value
    
    # 중간 출력: 엑셀에서 읽어온 데이터 출력
    print("Excel에서 읽어온 데이터:")
    print(excel_df)
    
    # ad_amount 컬럼명을 gc로 변경
    excel_df = excel_df.rename(columns={'ad_amount': 'gc'})
    
    # 두 DataFrame 병합 (아래로 붙이기)
    merged_df = pd.concat([df, excel_df], ignore_index=True)
    
    # 엑셀 파일 닫기
    wb.close()
    app.quit()
    
    return merged_df, excel_df  # Excel 데이터를 함께 반환

# Group by 하고 gc 합계 계산
def group_and_sum_gc(df):
    df_grouped = df.groupby('profit_center', as_index=False)['gc'].sum()
    return df_grouped

# 피벗 테이블 생성 및 ad 시트 추가 함수
def create_pivot_table(df, excel_df):
    # xlwings를 사용하여 피벗 테이블 생성
    app = xw.App(visible=True)
    wb = app.books.add()
    sheet = wb.sheets[0]
    
    # 데이터 삽입
    sheet.range('A1').value = df
    
    # 피벗 테이블 범위 설정
    pivot_sheet = wb.sheets.add('피벗테이블')
    data_range = sheet.range('A1').expand()
    
    # 피벗 테이블 생성
    pivot_cache = pivot_sheet.book.api.PivotCaches().Create(SourceType=xw.constants.PivotTableSourceType.xlDatabase,
                                                            SourceData=data_range.api)
    pivot_table = pivot_cache.CreatePivotTable(TableDestination=pivot_sheet.range('A3').api, TableName='PivotTable1')
    
    # 피벗 테이블 설정 (profit_center 기준, gc 합계)
    pivot_table.PivotFields('profit_center').Orientation = xw.constants.PivotFieldOrientation.xlRowField
    pivot_table.PivotFields('gc').Orientation = xw.constants.PivotFieldOrientation.xlDataField
    
    # ad 시트 추가 및 Excel 데이터 삽입
    ad_sheet = wb.sheets.add('ad')
    ad_sheet.range('A1').value = excel_df
    
    # 피벗 테이블 저장
#     wb.save('피벗_결과.xlsx')
#     app.quit()

# 메인 실행 함수
def main_process(end_date):
    conn = None
    try:
        # DB 연결
        conn = get_connection()
        
        # DB에서 필터링된 데이터 가져오기
        df = get_filtered_data(conn, end_date)
        
        # Profit center 값 변경
        df = update_profit_center(df)
        
        # Excel 파일과 병합
        merged_df, excel_df = merge_with_excel(df)
        
        # Group by 하고 gc 합계 계산
        grouped_df = group_and_sum_gc(merged_df)
        
        # 피벗 테이블 생성 및 ad 시트 추가
        create_pivot_table(grouped_df, excel_df)

    except Exception as e:
        print(f"에러 발생: {str(e)}")
    finally:
        if conn:
            conn.close()

# Tkinter GUI로 사용자 입력 받기
def run_gui():
    def process_data():
        input_date = entry_date.get()
        try:
            end_date = datetime.strptime(input_date, "%Y-%m-%d")
            main_process(end_date)
            messagebox.showinfo("성공", "작업이 완료되었습니다!")
        except ValueError:
            messagebox.showerror("오류", "잘못된 날짜 형식입니다. YYYY-MM-DD 형식으로 입력해주세요.")
    
    root = tk.Tk()
    root.title("날짜 입력")
    
    label_date = tk.Label(root, text="기준 날짜 (YYYY-MM-DD):")
    label_date.pack(pady=10)
    
    entry_date = tk.Entry(root)
    entry_date.pack(pady=10)
    
    btn_process = tk.Button(root, text="실행", command=process_data)
    btn_process.pack(pady=10)
    
    root.mainloop()

# GUI 실행
run_gui()


In [6]:
import pandas as pd
import xlwings as xw
from trino.dbapi import connect
from trino.auth import BasicAuthentication
import tkinter as tk
from tkinter import messagebox
from datetime import datetime

# Trino DB 연결 설정 함수
def get_connection():
    return connect(
        host='data-query.hec.co.kr',
        port=443,
        http_scheme='https',
        user='0900051',
        auth=BasicAuthentication('0900051', 'roseofpA1!!!!!'),
        catalog='dw',
        schema='fie'
    )

# DB에서 데이터 가져오기 함수
def get_filtered_data(conn, end_date):
    query = f"""
    SELECT v.gc, v.profit_center
    FROM s_accounting_ledger v
    WHERE v.acc_num BETWEEN 11530110 AND 11530199
      AND v.posting_date <= CAST('{end_date.strftime("%Y-%m-%d")}' AS DATE)
    """
    df = pd.read_sql(query, conn)
    df['계정'] = '매출채권'  # '계정' 칼럼 추가
    return df

# Profit center 값 변경 함수
def update_profit_center(df):
    replacements = {
        'AAID1903': 'AAID1902',
        'TBEMY1601': 'BEMY1601',
        'TAAMY1701': 'AAMY1701',
        'TAAID2102': 'AAID2102',
        'TAAID2101': 'AAID1902',
        'AAPL2101': 'TAAPL2101',
        'TBEPH1402': 'BEPH1402',
        'TBEID1401': 'BEID1401',
        'ANUS2201': 'TANUS2201',
        'AOUS2201': 'TAOUS2201',
        'ANID2201': 'ANID2202',
        'ANID2203': 'ANID2201'
    }
    
    df['profit_center'] = df['profit_center'].replace(replacements)
    return df

# Excel 파일과 병합하는 함수
def merge_with_excel(df):
    # 엑셀 파일 열기
    excel_path = 'refe.xlsx'  # 기본 폴더에 있는 파일 경로
    app = xw.App(visible=True)  # 엑셀을 눈에 보이도록 설정
    wb = app.books.open(excel_path)
    sheet = wb.sheets['Sheet1']
    
    # Excel 데이터 불러오기 (첫 줄을 헤더로 설정)
    excel_df = sheet.range('A1').options(pd.DataFrame, index=False, header=True, expand='table').value
    
    # '계정' 칼럼에서 '매출채권' 문자열 포함하는 데이터만 필터링
    excel_df = excel_df[excel_df['계정'].str.contains('매출채권', na=False)]
    
    # 중간 출력: 엑셀에서 읽어온 데이터 출력
    print("Excel에서 읽어온 데이터 (계정에 '매출채권' 포함):")
    print(excel_df)
    
    # ad_amount 컬럼명을 gc로 변경
    excel_df = excel_df.rename(columns={'ad_amount': 'gc'})
    
    # 두 DataFrame 병합 (아래로 붙이기)
    merged_df = pd.concat([df, excel_df], ignore_index=True)
    
    return merged_df, excel_df  # Excel 데이터를 함께 반환

# Group by 하고 gc 합계 계산
def group_and_sum_gc(df):
    df_grouped = df.groupby('profit_center', as_index=False)['gc'].sum()
    return df_grouped

# 피벗 테이블 생성 및 ad 시트 추가 함수
def create_pivot_table(df, excel_df):
    # xlwings를 사용하여 피벗 테이블 생성
    app = xw.App(visible=True)
    wb = app.books.add()
    sheet = wb.sheets[0]
    
    # 데이터 삽입
    sheet.range('b1').value = df
    
    # 피벗 테이블 범위 설정
    pivot_sheet = wb.sheets.add('피벗테이블')
    data_range = sheet.range('A1').expand()
    
    # 피벗 테이블 생성
    pivot_cache = pivot_sheet.book.api.PivotCaches().Create(SourceType=xw.constants.PivotTableSourceType.xlDatabase,
                                                            SourceData=data_range.api)
    pivot_table = pivot_cache.CreatePivotTable(TableDestination=pivot_sheet.range('A3').api, TableName='PivotTable1')
    
    # 피벗 테이블 설정 (profit_center 기준, gc 합계)
    pivot_table.PivotFields('profit_center').Orientation = xw.constants.PivotFieldOrientation.xlRowField
    pivot_table.PivotFields('gc').Orientation = xw.constants.PivotFieldOrientation.xlDataField
    
    # ad 시트 추가 및 Excel 데이터 삽입
    ad_sheet = wb.sheets.add('ad')
    ad_sheet.range('A1').value = excel_df
    
    # 파일 저장 없이 엑셀을 열린 상태로 유지
    app.visible = True

# 메인 실행 함수
def main_process(end_date):
    conn = None
    try:
        # DB 연결
        conn = get_connection()
        
        # DB에서 필터링된 데이터 가져오기
        df = get_filtered_data(conn, end_date)
        
        # Profit center 값 변경
        df = update_profit_center(df)
        
        # Excel 파일과 병합
        merged_df, excel_df = merge_with_excel(df)
        
        # Group by 하고 gc 합계 계산
        grouped_df = group_and_sum_gc(merged_df)
        
        # 피벗 테이블 생성 및 ad 시트 추가
        create_pivot_table(grouped_df, excel_df)

    except Exception as e:
        print(f"에러 발생: {str(e)}")
    finally:
        if conn:
            conn.close()

# Tkinter GUI로 사용자 입력 받기
def run_gui():
    def process_data():
        input_date = entry_date.get()
        try:
            end_date = datetime.strptime(input_date, "%Y-%m-%d")
            main_process(end_date)
            messagebox.showinfo("성공", "작업이 완료되었습니다!")
        except ValueError:
            messagebox.showerror("오류", "잘못된 날짜 형식입니다. YYYY-MM-DD 형식으로 입력해주세요.")
    
    root = tk.Tk()
    root.title("날짜 입력")
    
    label_date = tk.Label(root, text="기준 날짜 (YYYY-MM-DD):")
    label_date.pack(pady=10)
    
    entry_date = tk.Entry(root)
    entry_date.pack(pady=10)
    
    btn_process = tk.Button(root, text="실행", command=process_data)
    btn_process.pack(pady=10)
    
    root.mainloop()

# GUI 실행
run_gui()


In [8]:
import pandas as pd
import xlwings as xw
from trino.dbapi import connect
from trino.auth import BasicAuthentication
import tkinter as tk
from tkinter import messagebox
from datetime import datetime

# Trino DB 연결 설정 함수
def get_connection():
    return connect(
        host='data-query.hec.co.kr',
        port=443,
        http_scheme='https',
        user='0900051',
        auth=BasicAuthentication('0900051', 'roseofpA1!!!!!'),
        catalog='dw',
        schema='fie'
    )

# DB에서 데이터 가져오기 함수
def get_filtered_data(conn, end_date):
    query = f"""
    SELECT v.gc, v.profit_center
    FROM s_accounting_ledger v
    WHERE v.acc_num BETWEEN 11530110 AND 11530199
      AND v.posting_date <= CAST('{end_date.strftime("%Y-%m-%d")}' AS DATE)
    """
    df = pd.read_sql(query, conn)
    df['계정'] = '매출채권'  # '계정' 칼럼 추가
    return df

# Profit center 값 변경 함수
def update_profit_center(df):
    replacements = {
        'AAID1903': 'AAID1902',
        'TBEMY1601': 'BEMY1601',
        'TAAMY1701': 'AAMY1701',
        'TAAID2102': 'AAID2102',
        'TAAID2101': 'AAID1902',
        'AAPL2101': 'TAAPL2101',
        'TBEPH1402': 'BEPH1402',
        'TBEID1401': 'BEID1401',
        'ANUS2201': 'TANUS2201',
        'AOUS2201': 'TAOUS2201',
        'ANID2201': 'ANID2202',
        'ANID2203': 'ANID2201'
    }
    
    df['profit_center'] = df['profit_center'].replace(replacements)
    return df

# Excel 파일과 병합하는 함수
def merge_with_excel(df):
    # 엑셀 파일 열기
    excel_path = 'refe.xlsx'  # 기본 폴더에 있는 파일 경로
    app = xw.App(visible=True)  # 엑셀을 눈에 보이도록 설정
    wb = app.books.open(excel_path)
    sheet = wb.sheets['Sheet1']
    
    # Excel 데이터 불러오기 (첫 줄을 헤더로 설정, 인덱스 없이 읽기)
    excel_df = sheet.range('A1').options(pd.DataFrame, index=False, header=True, expand='table').value
    
    # '계정' 칼럼에서 '매출채권' 문자열 포함하는 데이터만 필터링
    excel_df = excel_df[excel_df['계정'].str.contains('매출채권', na=False)]
    
    # 중간 출력: 엑셀에서 읽어온 데이터 출력
    print("Excel에서 읽어온 데이터 (계정에 '매출채권' 포함):")
    print(excel_df)
    
    # ad_amount 컬럼명을 gc로 변경
    excel_df = excel_df.rename(columns={'ad_amount': 'gc'})
    
    # 두 DataFrame 병합 (아래로 붙이기)
    merged_df = pd.concat([df, excel_df], ignore_index=True)
    
    # 엑셀 파일 닫기
    return merged_df, excel_df  # Excel 데이터를 함께 반환

# Group by 하고 gc 합계 계산
def group_and_sum_gc(df):
    df_grouped = df.groupby('profit_center', as_index=False)['gc'].sum()
    return df_grouped

# 피벗 테이블 생성 및 ad 시트 추가 함수
# 피벗 테이블 생성 및 ad 시트 추가 함수
def create_pivot_table(df, excel_df):
    # xlwings를 사용하여 피벗 테이블 생성
    app = xw.App(visible=True)
    wb = app.books.add()
    sheet = wb.sheets[0]
    
    # 인덱스를 제거하고 데이터 삽입
    sheet.range('A1').value = df.to_numpy()  # 인덱스 없이 데이터 삽입
    sheet.range('A1').value = df.columns.tolist()  # 헤더 추가
    
    # 피벗 테이블 범위 설정
    data_range = sheet.range('A1').expand()
    
    # 피벗 테이블 생성 위치 설정
    pivot_sheet = wb.sheets.add('피벗테이블')
    pivot_cache = pivot_sheet.book.api.PivotCaches().Create(SourceType=xw.constants.PivotTableSourceType.xlDatabase,
                                                            SourceData=data_range.api)
    pivot_table = pivot_cache.CreatePivotTable(TableDestination=pivot_sheet.range('A3').api, TableName='PivotTable1')
    
    # 피벗 테이블 설정 (profit_center 기준, gc 합계)
    pivot_table.PivotFields('profit_center').Orientation = xw.constants.PivotFieldOrientation.xlRowField
    pivot_table.PivotFields('gc').Orientation = xw.constants.PivotFieldOrientation.xlDataField
    
    # ad 시트 추가 및 Excel 데이터 삽입
    ad_sheet = wb.sheets.add('ad')
    ad_sheet.range('A1').value = excel_df

    # 파일 저장 없이 엑셀을 열린 상태로 유지
    app.visible = True


# 메인 실행 함수
def main_process(end_date):
    conn = None
    try:
        # DB 연결
        conn = get_connection()
        
        # DB에서 필터링된 데이터 가져오기
        df = get_filtered_data(conn, end_date)
        
        # Profit center 값 변경
        df = update_profit_center(df)
        
        # Excel 파일과 병합
        merged_df, excel_df = merge_with_excel(df)
        
        # Group by 하고 gc 합계 계산
        grouped_df = group_and_sum_gc(merged_df)
        
        # 피벗 테이블 생성 및 ad 시트 추가
        create_pivot_table(grouped_df, excel_df)

    except Exception as e:
        print(f"에러 발생: {str(e)}")
    finally:
        if conn:
            conn.close()

# Tkinter GUI로 사용자 입력 받기
def run_gui():
    def process_data():
        input_date = entry_date.get()
        try:
            end_date = datetime.strptime(input_date, "%Y-%m-%d")
            main_process(end_date)
            messagebox.showinfo("성공", "작업이 완료되었습니다!")
        except ValueError:
            messagebox.showerror("오류", "잘못된 날짜 형식입니다. YYYY-MM-DD 형식으로 입력해주세요.")
    
    root = tk.Tk()
    root.title("날짜 입력")
    
    label_date = tk.Label(root, text="기준 날짜 (YYYY-MM-DD):")
    label_date.pack(pady=10)
    
    entry_date = tk.Entry(root)
    entry_date.pack(pady=10)
    
    btn_process = tk.Button(root, text="실행", command=process_data)
    btn_process.pack(pady=10)
    
    root.mainloop()

# GUI 실행
run_gui()
